# 情報科学演習II アルゴリズムとデータ構造II 第3週 1冊目

情報科学演習IIの前半3週間はアルゴリズムとデータ構造IIのテーマです．
このテーマで使うプログラミング言語はPythonです．

アルゴリズムとデータ構造IIの第3週のノートブックはこの1冊です．
ダイクストラのアルゴリズムの複数の実装の実行時間を比べてください．


#### 第3週目次

1. 先週の実装を用意する
2. 三つの実装の実行時間を比べる
3. 両対数グラフを知る

## 1. 先週の実装を用意する

実行時間を比べる実装は次の3つです．
- 先週の`FirstImplementation`クラス
- 先週の`SecondImplementation`クラス（できていない人はなくてよい）
- networkxの[single_source_dijkstra_path_length](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.weighted.single_source_dijkstra_path_length.html)関数

このノートブックに，先週ダイクストラのアルゴリズムを実装した`FirstImplementation`クラスと`SecondImplementation`クラスをこのノートブックの以下の二つのコードセルにコピーして実行できるようにします．

次の3つのセルは準備ですので，実行してください．
最後ののセルで必要なモジュールを`import`し，いくつかの関数を定義しておきます．

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz
# 最終行は Successfully installed pygraphviz-x.xx
# x.xxの部分は1.13などのバージョン番号

In [ ]:
# グラフの準備
!git clone https://github.com/hu-ids-exercise-algorithm-II/graphs.git

In [ ]:
# 準備
import networkx as nx
import matplotlib.pyplot as plt
import math
import pygraphviz as pgv
from IPython.display import SVG, display_svg
from test import support # sortdict関数
import time

log = False # log = Trueにすると途中経過が出力される

def print_graph(G): # GはnetworkxのGraphクラス (networkx.classes.graph.Graphクラス)
  print('print_graph:')
  print("  nodes", list(G.nodes))
  # print('  edges', list(G.edges))
  print('  edges', list(G.edges.data())) # 辺の重みも出力

def draw_graph(G, filename = 'tmp'):
  if G.order() > 100:
    print(F'draw_graph: canceled for too much nodes ({G.order()} > 100)')
    return
  A = nx.nx_agraph.to_agraph(G)
  for edge in G.edges(data="weight"):
    u, v, d = edge
    A.get_edge(u, v).attr['label'] = d

  A.layout(args='-Nshape=circle')  # default prog='neato' 
  A.draw(F'{filename}.svg')
  display_svg(SVG(F'{filename}.svg'))

def sample_graph1():
  G = nx.Graph() # 無向グラフ
  for i,j,w in [[0, 1, 5], [0, 4, 9], [0, 7, 8], [1, 2, 12], [1, 3, 15], 
                [1, 7, 4], [2, 3, 3], [2, 6, 11], [3, 6, 9], [4, 5, 4], 
                [4, 7, 5], [5, 2, 1], [5, 6, 13], [7, 2, 7], [7, 5, 6]]:
    G.add_edge(i, j, weight=w) # weightはnetworkx, labelはgraphviz
  w = nx.get_edge_attributes(G, 'weight')
  print('edge weight', w)
  return G

G1 = sample_graph1()
w1 = nx.get_edge_attributes(G1, 'weight')
answer_d1 = {0: 0, 1: 5, 2: 14, 3: 17, 4: 9, 5: 13, 6: 25, 7: 8}

`SecondImplementation`クラスができなかった人は，`SecondImplementation`クラス以外の実行時間を測ってください．

**練習**
次のセルに`FirstImplementation`クラスをコピーしなさい．

In [ ]:
# FirstImplementationクラス
class FirstImplementation:
  ....

**練習**
次のセルを実行して`FirstImplementation`クラスをテストしなさい．
先週と同じテストです．
求めた距離が間違っていたら実装に失敗しています．

In [ ]:
f = True
for s in range(G1.order()):
  d = FirstImplementation().dijkstra(G1, w1, s)
  answer_d = nx.single_source_dijkstra_path_length(G1, s)
  if d != answer_d:
    f = False
    print(F'始点{s}のとき間違っています')
    print('求めた距離', support.sortdict(d))
    print('正しい距離', support.sortdict(answer_d))
    for u in G1.nodes():
      if d[u] != answer_d[u]:
        print(F'頂点{s}から頂点{u}への最短距離は{answer_d[u]}ですが，{d[u]}と求めています．')
    print()
if f:
  print(F'{G1.order()}個の始点すべてにおいて正しく距離を求めています．')
else:
  print('距離が間違っている場合があります．')

**練習**
次のセルに`SecondImplementation`クラスをコピーしなさい．

In [ ]:
# SecondImplementationクラス
class SecondImplementation:
  ....

**練習**
次のセルを実行して`SecondImplementation`クラスをテストしなさい．
先週と同じテストです．
求めた距離が間違っていたら実装に失敗しています．

In [ ]:
f = True
for s in range(G1.order()):
  d = SecondImplementation().dijkstra(G1, w1, s)
  answer_d = nx.single_source_dijkstra_path_length(G1, s)
  if d != answer_d:
    f = False
    print(F'始点{s}のとき間違っています')
    print('求めた距離', support.sortdict(d))
    print('正しい距離', support.sortdict(answer_d))
    for u in G1.nodes():
      if d[u] != answer_d[u]:
        print(F'頂点{s}から頂点{u}への最短距離は{answer_d[u]}ですが，{d[u]}と求めています．')
    print()
if f:
  print(F'{G1.order()}個の始点すべてにおいて正しく距離を求めています．')
else:
  print('距離が間違っている場合があります．')

## 2. 三つの実装の実行時間を比べる

様々なグラフに対して三つの実装の実行時間を計測します．

### 2.1 実行時間の測り方

実行時間の測り方は第1週の5章と同じです．
最短距離を求める前後に現在の時刻を取得し，その差を実行時間とします．
現在の時刻は`time.time()`関数で取得します．

`time.sleep(1.0)`の実行時間は次のセルのように`sleep`関数の前後の時刻から求めます．

In [ ]:
t0 = time.time()
time.sleep(0.1)
t1 = time.time()
t1 - t0 # 時間の単位は秒

**練習**
`time.sleep(1.0)`の実行時間を5回測り，その中央値を取り出すように次のセルの`....`の部分を書きなさい．

- `for`の中に上のセルの4行を追加します．
- 4行のうち最後の行は，求めた実行時間`t1 - t0`をリスト`t`に追加するように書き換えます．

  `t.append(t1 - t0)`

In [ ]:
n = 5  # 実行回数（中央値を取るため奇数）
t = [] # 1回ずつの実行時間を格納するリスト
for i in range(5):
  # ここに実行時間を測るコードを書く
  ....
print('すべての実行時間', t)
t.sort()  # リストtを短い順に並べ替える
t[n // 2] # // は整数除算

### 2.2 実行時間を測るグラフを用意する

次のセルの`cases`はグラフと始点の組のリストです．
例えば，リスト`cases`の最初の要素は`['graphs/Delaunary-5-exp-10.edgelist', 0]`です．
リストの入れ子なので，この要素もリストで，
`'graphs/Delaunary-5-exp-10.edgelist'`の部分がファイル名で，0は最短距離を求める際の始点にしています．

ファイル名のDulaunaryの後の数が頂点数です．
次のセルのif文の条件をFalseからTrueに変更すると，グラフの頂点数，辺の数，グラフの図を出力できます．

In [ ]:
cases = [['graphs/Delaunary-5-exp-10.edgelist', 0],
         ['graphs/Delaunary-10-exp-10.edgelist', 0],
         ['graphs/Delaunary-20-exp-10.edgelist', 0],
         ['graphs/Delaunary-50-exp-10.edgelist', 0],
         ['graphs/Delaunary-100-exp-10.edgelist', 0],
         ['graphs/Delaunary-200-exp-10.edgelist', 0],
         ['graphs/Delaunary-500-exp-10.edgelist', 0],
         ['graphs/Delaunary-1000-exp-10.edgelist', 0],
         ['graphs/Delaunary-2000-exp-10.edgelist', 0],
         ['graphs/Delaunary-5000-exp-10.edgelist', 0],
         ['graphs/Delaunary-10000-exp-10.edgelist', 0],
         ['graphs/Delaunary-20000-exp-10.edgelist', 0],
         ['graphs/Delaunary-50000-exp-10.edgelist', 0],
         ['graphs/Delaunary-100000-exp-10.edgelist', 0]]

if False: # Trueにするとそれぞれのファイルのグラフのの頂点数などを出力します．
  for edgelist, s in cases:
    G = nx.read_edgelist(edgelist, nodetype=int)
    print(F'頂点数 {G.order()}, 辺の数 {G.size()}, ファイル名 {edgelist}')
    draw_graph(G)

次のコードのように`cases.pop()`で実行時間を測るグラフを減らすことができます．

計測対象のグラフを減らす場合はリスト`cases`から要素を削除してください．
`cases.pop()`で末尾の要素が，`cases.pop(i)`で`i`番目の要素が削除されます．
なお，`pop`関数の使い方は第1週の3冊目で確認できます．

In [ ]:
# 削除したいケースをここで削除してください．
cases.pop() # ノード数が10万のグラフを計測対象から削除 （Delaunary-100000)
cases.pop() # ノード数が5万のグラフを計測対象から削除 (Delaunary-50000)
cases.pop() # ノード数が2万のグラフを計測対象から削除 (Delaunary-20000)
# cases.pop() # ノード数が1万のグラフを計測対象から削除 (Delaunary-10000)
# cases.pop() # ノード数が5千のグラフを計測対象から削除 (Delaunary-5000)
# cases.pop() # ノード数が2万のグラフを計測対象から削除 (Delaunary-2000)
cases

### 2.3 実行時間を計測する

それでは実行時間を計測しましょう．
実行に必要なコードは書いてあります．

実行時間を記録する`measurements`という辞書を用意しています．
辞書のキーは3つの実装を区別する文字列`first`, `second`, `networkx`とします．
後のコードでこのキーによって最短経路問題を解くダイクストラのアルゴリズムの実装を切り替えます．
このキーはグラフの系列名としても使います．

この辞書の値は二つの配列の配列にしています．
一つ目の配列にはグラフの頂点数を，二つ目の配列には実行時間を記録します．

`SecondImplementation`を使わない場合は
`measurements['second']`で始まる行を次のセルから削除しなさい．
あるいはその行の先頭に#をつけてコメントアウトしなさい．

In [ ]:
measurements = {}
measurements['networkx'] = [[], []]
measurements['first'] = [[], []]
measurements['second'] = [[], []]
measurements

In [ ]:
tstart = time.time()
for label in list(measurements): # キー（label）を一つずつ取り出す．
  print(F'実装 {label}')
  for edgelist, s in cases:
    G = nx.read_edgelist(edgelist, nodetype=int)
    w = nx.get_edge_attributes(G, 'weight')
    print(F'ファイル名 {edgelist}, 始点 {s}, 頂点数 {G.order()}, 辺の数 {G.size()}')
    ntries = 5 # 試行回数
    tlist = [] # 試行ごとの実行時間
    for i in range(ntries):
      match label:
        case 'networkx':
          t0 = time.time()
          d0 = nx.single_source_dijkstra_path_length(G, s)
          t1 = time.time()
        case 'first':
          t0 = time.time()        
          c1 = FirstImplementation()
          d1 = c1.dijkstra(G, w, s)
          t1 = time.time()
        case 'second':
          t0 = time.time()
          c2 = SecondImplementation()
          d1 = c2.dijkstra(G, w, s)
          t1 = time.time()

      tlist.append(t1 - t0)
      if False: # 途中経過を出力したい場合はTrueにする
        print('tlist', tlist)
    tlist.sort()
    measurements[label][0].append(G.order())
    measurements[label][1].append(tlist[ntries // 2])
    print('このセルを実行してからの経過時間[秒]', time.time() - tstart)

### 2.4 実行時間のグラフを作成する

計測した実行時間は`measurements`に格納している．
次のセルを実行して内容を確認しよう．

In [ ]:
measurements

In [ ]:
# 辞書型のmeasurementsのキーを出力する
list(measurements)

In [ ]:
plt.figure(figsize=[5, 4]) # グラフのサイズ
for label, xy in measurements.items():
   x, y = xy
   ymicro = [t * 10**6 for t in y] # 計測時間の単位を秒からマイクロ秒に変換
   plt.plot(x, ymicro, marker='.', label=label)
plt.xlabel('number of nodes') # X軸ラベル
plt.ylabel('time [μ second]')   # Y軸ラベル 
plt.xscale('log')                # X軸を対数軸に
plt.yscale('log')                # Y軸を対数軸に
plt.legend(loc='upper left')     # 凡例
plt.show()

この計測結果を分析してレポートを作成しなさい．

## 3. 両対数グラフを知る

両対数グラフに慣れていないと思いますので補足します．
この章の内容を理解すると計測結果の分析がしやすくなるはずです．
両対数グラフはX軸もY軸も目盛りを対数にしたグラフです．

対数軸では，1と10の差と10と100の差，20と200の差がどちらも同じ長さ1で表現されます．
次の式のようにどちらも対数($\log$)の差にすると1だからです．
$$\log_{10}\, 10 - \log_{10}\, 1 = 1 - 0 = 1$$
$$\log_{10}\, 100 - \log_{10}\, 10 = 2 - 1 = 1$$
$$\log_{10}\, 200 - \log_{10}\, 20 = 2.301\ldots - 1.301\ldots = 1$$

次のセルを実行すると$y=x$, $y=10x$, $y=100x$の3つの関数を両対数グラフに出力します．
$x$の値として13個選んでそれらの間は直線で近似しています．
直線$y=ax$の傾き$a$の部分が変化しても両対数グラフでは傾きが同じ直線で，
$a$の部分は直線がY軸の方向に$\log_{10}a$だけ移動していることがわかります．

`plt.yscale('log')`の部分をコメントアウトするか引数を`'linear'`に書き換えると，Y軸が線形軸になります．
X軸も同様に対数軸と線形軸を切り替えられます．

**練習**
次のセルを実行すると，同じグラフが二つ出力されています．
二つ目のグラフを線形軸に変更しなさい．
同じ一次関数が線形軸と対数軸でどのように違うグラフになるか確認しなさい．
X軸とY軸の一方を対数軸，他方を線形軸にした場合も試しなさい．

In [ ]:
x = list(map(lambda x: 10**(x / 3.0), range(0 ,13)))
print('x', x) # ノード数のリスト

# y = x, 実行時間[μs]がノード数になる場合
y1 = x
print('y = x', y1) # 実行時間ののリスト

# y = 10x, 実行時間がノード数の10倍になる場合
def x10(x): return 10 * x
y10 = list(map(lambda x: x10(x), x))
print('y = 10x', y10) # 実行時間ののリスト

# y = 100x, 実行時間がノード数の100倍になる場合
def x100(x): return 100 * x
y100 = list(map(lambda x: x100(x), x))
print('y = 100x', y100) # 実行時間ののリスト

# 一つ目の目のグラフ
plt.figure(figsize=[5, 3]) # グラフのサイズ
plt.plot(x, y1, marker='.', label='y = x')
plt.plot(x, y10, marker='.', label='y = 10x')
plt.plot(x, y100, marker='.', label='y = 100x')
plt.xscale('log')                # X軸は対数軸(log)．線形軸はlinear
plt.yscale('log')                # Y軸は対数軸(log)．線形軸はlinear
def show(plt):
  plt.xlabel('number of nodes')  # X軸ラベル
  plt.ylabel('time [μ second]')  # Y軸ラベル 
  plt.legend(loc='upper left')   # 凡例
  plt.show()
show(plt)

# 二つ目のグラフ
plt.figure(figsize=[5, 3]) # グラフのサイズ
plt.plot(x, y1, marker='.', label='y = x')
plt.plot(x, y10, marker='.', label='y = 10x')
plt.plot(x, y100, marker='.', label='y = 100x')
plt.xscale('log')                # X軸は対数軸(log)．線形軸はlinear
plt.yscale('log')                # Y軸は対数軸(log)．線形軸はlinear
show(plt)

次は$a$次関数を対数軸のグラフにしてみましょう．
次のコードは$y=x^1$, $y=x^2$, $y=x^3$, $y=x^4$の4つの関数をグラフを出力します．
この二つのグラフを出力しますが，二つとも線形軸のグラフです．
$y=x^a$の$a$次関数は次数が増えると線形グラフだと急なカーブを描きます．

**練習**
一つ目のグラフを両対数グラフにしなさい．二つ目のグラフの軸は整形軸のままでよい．前のセルと同様に`plt.xscale('log')`と`plt.yscale('log')`にすると両対数グラフになる．

In [ ]:
x = list(map(lambda x: 10**(x / 10.0), range(0, 11)))
print('x', x)

# y = x
yp1 = x
print('y = x', yp1)

# y = x * x
def pow2(x): return x * x
yp2 = list(map(lambda x: pow2(x), x))
print('y = x * x', yp2)

# y = x^3
def pow3(x): return x * x * x
yp3 = list(map(lambda x: pow3(x), x))
print('y = x^3', yp3)

# y = x^4
def pow4(x): return x ** 4
yp4 = list(map(lambda x: pow4(x), x))
print('y = x^4', yp4)

y_label = [[yp1, 'y = x'], [yp2, 'y = x * x'], [yp3, 'y = x^3'], [yp4, 'y = x^4']]

# 一つ目のグラフ
plt.figure(figsize=[5, 3]) # グラフのサイズ
for y, label in y_label:
  plt.plot(x, y, marker='.', label=label)
plt.xscale('linear') # X軸は対数軸(log)．線形軸はlinear
plt.yscale('linear') # Y軸は対数軸(log)．線形軸はlinear
show(plt)

# 二つ目のグラフ
plt.figure(figsize=[5, 3]) # グラフのサイズ
for y, label in y_label:
  plt.plot(x, y, marker='.', label=label)
show(plt)

**練習**
上のコードを複製して，$x$の範囲を$1\le x\le 10$から$1\le x\le 10^4=10000$に変更しなさい．
コード先頭の`range(0, 11)`の`11`を`41`に変更すると$x\le 10^{40}$に変更できる．
一つ目のグラフのX,Y軸は対数軸にしなさい．
二つ目のグラフのX,Y軸は線形軸にしなさい．

**練習**
上のグラフで$y=x$や$y=x^2$の線がどこにあるか考えて次のテキストセルに書きなさい．

解答欄：

最後に指数関数も両対数グラフに表してみる．
グラフにする関数は$1\le x\le 100$の範囲で$y=x$, $y=1000x$, $y=x^{10}$, $y=2^x$の4つの関数のグラフを示す．

**練習**
次のコードで出力するグラフを軸を両対数にして4つの関数の傾向を見なさい．
傾向をコードセルの下のテキストセルに書きなさい．

In [ ]:
# 指数
x = list(map(lambda x: 10**(x / 10.0), range(0, 21)))

# y = x
y1 = x
y_label = [[y1, 'y = x']]

# y = 1000x
def x1000(x): return 1000 * x
y1000 = list(map(lambda x: x1000(x), x))
y_label.append([y1000, 'y = 1000x'])

# y = x^10
def pow10(x): return x ** 10
yp10 = list(map(lambda x: pow10(x), x))
y_label.append([yp10, 'y = x^10'])

# y = 2^x
def exp10(x): return 2 ** x
ye2 = list(map(lambda x: exp10(x), x))
y_label.append([ye2, 'y = 2^x'])

# グラフ
plt.figure(figsize=[5, 3]) # グラフのサイズ
for y, label in y_label:
  plt.plot(x, y, marker='.', label=label)
plt.xscale('linear')             # X軸 logなら対数軸 linearなら線形軸
plt.yscale('linear')             # Y軸 logなら対数軸 linearなら線形軸
show(plt)

解答欄：

2.4節で作成した実行時間のグラフは両対数グラフです．
改めて実行時間のグラフを見て，実行時間がノード数に対して線形に増加しているのか，$a>1$の$a$次関数で増加してるのか考えてください．

ノートブックは以上です．
レポートを作成してください．